# Inheritance: For Good or For Worse
Many consider multiple inheritance more trouble than it’s worth. Java does not provide support for multiple inheritance in classes. The lack of it certainly did not hurt Java; it probably fueled its widespread adoption after many were traumatized by the excessive use of multiple inheritance in C++.

## Subclassing Built-In Types Is Tricky
Subclassing built-in types like dict or list or str directly is errorprone because the built-in methods mostly ignore user-defined overrides. Instead of subclassing the built-ins, derive your classes from the collections module using UserDict, UserList, and UserString, which are designed to be easily extended.

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2) 


In [8]:
dd = DoppelDict(one=1) # __init__ method inherited from dict clearly ignored that __setitem__ was overridden
dd

{'one': 1}

In [9]:
dd['two'] = 2 # The [] operator calls our __setitem__ and works as expected
dd

{'one': 1, 'two': [2, 2]}

In [10]:
dd.update(three=3) # this also ignores our override.
dd

{'one': 1, 'two': [2, 2], 'three': 3}

We can fix this if we subclass collections.UserDict instead of dict.

In [11]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)


In [13]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [14]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [15]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

All is now working as expected.

## Multiple Inheritance and Method Resolution Order
Any language implementing multiple inheritance needs to deal with potential naming conflicts when unrelated ancestor classes implement a method by the same name. This is called the 'diamond problem'.

In [16]:
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [17]:
d = D()

In [18]:
d.pong()

pong: <__main__.D object at 0x00000241C3B037F0>


In [19]:
C.pong(d)

PONG: <__main__.D object at 0x00000241C3B037F0>


The ambiguity of a call like d.pong() is resolved because Python follows a specific order when traversing the inheritance graph. That order is called MRO: Method Resolution Order. Classes have an attribute called __mro__ holding a tuple of references to the
superclasses in MRO order.

In [20]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [21]:
d.ping()

ping: <__main__.D object at 0x00000241C3B037F0>
post-ping: <__main__.D object at 0x00000241C3B037F0>


We could remove this ambiguity and call A.ping(self) instead of super().ping(). However, it’s safest and more future-proof to use super(), especially when calling methods on a framework.

In [22]:
d = D()

In [23]:
d.pingpong()

ping: <__main__.D object at 0x00000241C3B03A20>
post-ping: <__main__.D object at 0x00000241C3B03A20>
ping: <__main__.D object at 0x00000241C3B03A20>
pong: <__main__.D object at 0x00000241C3B03A20>
pong: <__main__.D object at 0x00000241C3B03A20>
PONG: <__main__.D object at 0x00000241C3B03A20>


The MRO takes into account not only the inheritance graph but also the order in which superclasses are listed in a subclass declaration. In other words, if in diamond.py (Example 12-4) the D class was declared as class D(C, B):, the __mro__ of class Dwould be different: C would be searched before B.

## Some Mixin Examples...
Here are two main situations where mixins are used:
-  You want to provide a lot of optional features for a class.
-  You want to use one particular feature in a lot of different classes.

In [1]:
class HasMethod1(object):
    def method(self):
        return 1

class HasMethod2(object):
    def method(self):
        return 2

class UsesMethod10(object):
    def usesMethod(self):
        return self.method() + 10

class UsesMethod20(object):
    def usesMethod(self):
        return self.method() + 20

class C1_10(HasMethod1, UsesMethod10): pass
class C1_20(HasMethod1, UsesMethod20): pass
class C2_10(HasMethod2, UsesMethod10): pass
class C2_20(HasMethod2, UsesMethod20): pass

assert C1_10().usesMethod() == 11
assert C1_20().usesMethod() == 21
assert C2_10().usesMethod() == 12
assert C2_20().usesMethod() == 22

# Nothing prevents implementing the method
# on the base class like in Definition 1:

class C3_10(UsesMethod10):
    def method(self):
        return 3

assert C3_10().usesMethod() == 13

## Coping with Multiple Inheritance
It’s easy to create incomprehensible and brittle designs using multiple inheritance. Because we don’t have a comprehensive theory, here are a few tips to avoid spaghetti class graphs.
-  Distinguish Interface Inheritance from Implementation Inheritance. Inheritance of interface creates a subtype, implying an 'is-a' relationship. Inheritance of implementation avoids code duplication by reuse.
-  Make Interfaces Explicit with ABCs.
-  Use Mixins for Code Reuse.
-  Make Mixins Explicit by Naming. It is highly recommended that they are named with a …Mixin suffix.
-  An ABC May Also Be a Mixin; The Reverse Is Not True.
-  Don’t Subclass from More Than One Concrete Class.
-  Provide Aggregate Classes to Users.
-  Favor Object Composition Over Class Inheritance.

***